In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize
import torch

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')



# load data

In [ ]:
os.listdir(os.path.join(DATAPATH, 'dl'))

In [ ]:
data = os.path.join(PATH, 'datasets/data/dl', '211215_dl_classification_84_25_2cm_slice1_sample1x8192.h5')
h5datafile = h5py.File(data, 'r')
print(h5datafile['train']['data'].shape)

pitch_angles = h5datafile['train']['meta']['theta_min'][:]
energies = h5datafile['train']['meta']['energy'][:]

real = h5datafile['train']['data'][:, 0, :]
imag = h5datafile['train']['data'][:, 1, :]

cmplx_data = real + 1j * imag

# use metadata to select templates from a specific energy

In [ ]:
theta_min = 84
theta_max = 90

energy = 18575

indices = np.logical_and(pitch_angles >= theta_min, pitch_angles <= theta_max)

indices = np.logical_and(indices, energies == energy)

templates = cmplx_data[indices, :]

print(templates.shape)

In [ ]:
plt.plot(abs(templates[120, :]))

# Check score for identical pitch angle but different energy

In [ ]:
theta_min = 85
energy = 18600

index_signal = np.logical_and(pitch_angles == theta_min, energies == energy)

theta_min = 85
energy = 18575

index_template = np.logical_and(pitch_angles == theta_min, energies == energy)



signal = cmplx_data[index_signal, :]
template = cmplx_data[index_template, :]


plt.plot(abs(template[0, :]))
plt.plot(abs(signal[0, :]))

In [ ]:
print(abs(np.vdot(template, signal)))
print(np.max(abs(scipy.signal.correlate(template, signal))))

# normalize the templates, create tensor

In [ ]:
var = 60 * 10 * 50 * 200e6 * 1.38e-23 / 8192

norm = 1 / np.sqrt(var * (abs(templates) ** 2).sum(-1))

norm_templates = norm.reshape((norm.size, 1)) * templates

temp = np.zeros((121, 2, 8192))
temp[:, 0, :] = norm_templates.real
temp[:, 1, :] = norm_templates.imag

template_tensor = torch.as_tensor(temp)

temp = np.zeros((121, 2, 8192))
temp[:, 0, :] = templates.real
temp[:, 1, :] = templates.imag

signal_tensor = torch.as_tensor(temp)

In [ ]:
abs(np.vdot(norm_templates[20, :], templates[20, :]))

# define torch convolutional layer, initialize with the templates

In [ ]:
torch_conv = torch.nn.Conv1d(2, 121, 8192, padding_mode='circular', padding='same', bias=False)
torch_maxpool = torch.nn.MaxPool1d(8192)

In [ ]:
torch_conv.weight = torch.nn.Parameter(template_tensor, requires_grad=False)


In [ ]:
torch_conv.cuda()
torch_maxpool.cuda()

In [ ]:
convolution = torch_maxpool(torch_conv(signal_tensor.cuda()))


In [ ]:
convolution.shape

In [ ]:
plt.figure(figsize=(13, 8))

plt.imshow(abs(convolution.cpu().numpy()).max(axis=-1), aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
plt.plot(convolution.cpu().numpy()[20, 20, :])

In [ ]:
print(signal_tensor.shape)
print(template_tensor.shape)

In [ ]:
plt.plot(scipy.signal.correlate(signal_tensor.numpy()[100, 0, :], template_tensor.numpy()[100, 0, :]))
plt.plot(scipy.signal.correlate(signal_tensor.numpy()[100, 1, :], template_tensor.numpy()[100, 1, :]))


In [ ]:

plt.plot(
    scipy.signal.fftconvolve(np.flip(signal_tensor.numpy()[20, 0, :]), template_tensor.numpy()[20, 0, :], mode='same') + 
    scipy.signal.fftconvolve(np.flip(signal_tensor.numpy()[20, 1, :]), template_tensor.numpy()[20, 1, :], mode='same')
)

In [ ]:
os.listdir(os.path.join(RESULTPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211027_84_25_2cm.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

# compute scores

# load meta data

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)
#test_metadata = pd.DataFrame(test_data.metadata)

In [ ]:
energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

print(sorting_indexes.shape)

In [ ]:
print(unique_energy, unique_angle)

# remove grad-b shift, do beamforming, signal 1

In [ ]:
signal_energy = 18595
signal_angle = 89.5 

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192 * 3))


ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)
grad_b_freq = 2.5e3

shifted_signal = GradBShift(signal, 0.02, 0.00, -grad_b_freq)
#shifted_signal = GradBShift(signal, 0.02, 0.00, grad_b_freq)

#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

sum_signal = ShiftAndSum(shifted_signal, x_pos_range, y_pos_range, 0)

In [ ]:
sns.set_theme(style='ticks', context='paper')
cmap = sns.color_palette('mako_r', as_cmap=True)
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(1,1,1)

image = np.flip((abs(sum_signal) ** 2).sum(axis=-1).reshape((ngrid, ngrid)), axis=1)
print(image.max())

img = ax.imshow(image, cmap = cmap, aspect='auto', interpolation='none')

cbar = fig.colorbar(img)

# choose signal and template, apply grad-b shift to signal, check matched filter scores

In [ ]:
####

var = 1.38e-23 * 10 * 50 * 200e6
rng = np.random.default_rng()
noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3 )
noise = noise[:, 0] + 1j * noise[:, 1]

signal_energy = 18595
signal_angle = 89.6

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]
signal = template_data.data[index, :].reshape((60, 8192 * 3))

template_energy = 18595
template_angle = 89.6

index = template_metadata[(template_metadata['energy'] == template_energy) & (template_metadata['theta_min'] == template_angle)].index[0]
template = template_data.data[index, :].reshape((60, 8192 * 3))

template = (template.flatten() * 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))).reshape((60, 8192 * 3))

nlist = 301
grad_b_freq_list = np.linspace(-2e3, 2e3, nlist)
score_list_1 = np.zeros(nlist)

for i, freq in enumerate(grad_b_freq_list):
    
    noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    shift_signal = GradBShift(signal, 0.02, 0.00, -freq)
    
    score = abs(np.vdot(template.flatten(), shift_signal.flatten() + noise ))
    
    score_list_1[i] = score
    
    #print(i + 1)
    
####

template_energy = 18584
template_angle = 89.85

index = template_metadata[(template_metadata['energy'] == template_energy) & (template_metadata['theta_min'] == template_angle)].index[0]
template = template_data.data[index, :].reshape((60, 8192 * 3))

template = (template.flatten() * 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))).reshape((60, 8192 * 3))

score_list_2 = np.zeros(nlist)

for i, freq in enumerate(grad_b_freq_list):
    
    noise = rng.multivariate_normal([0, 0], np.eye(2) * var / 2, 60 * 8192 * 3)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    shift_signal = GradBShift(signal, 0.02, 0.00, -freq)
    
    score = abs(np.vdot(template.flatten(), shift_signal.flatten() + noise))
    
    score_list_2[i] = score

####

In [ ]:
sns.set_theme(style='whitegrid', context='paper')
clist = sns.color_palette('deep')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_1 - score_list_1.min(), '.')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_2 - score_list_2.min(), '.')

#ax.set_xlim(75, 125)
#ax.set_ylim(7, 8)

# fit a polynomial to the different curves to estimate the central position

In [ ]:
def fit_func(x, a, b, c, d):
    
    return d * np.exp(- (x - a) ** 2 / b ** 2) + c

popt_1, pcov_1 = scipy.optimize.curve_fit(fit_func, grad_b_freq_list, score_list_1-score_list_1.min(), [0.0, 1500, 0.0, 10])
popt_2, pcov_2 = scipy.optimize.curve_fit(fit_func, grad_b_freq_list, score_list_2-score_list_2.min(), [0.0, 1500, 0.0, 10])
print(popt_1, popt_2)
print(np.sqrt(pcov_1))
print(np.sqrt(pcov_2))

In [ ]:
sns.set_theme(style='whitegrid', context='paper')
clist = sns.color_palette('deep')

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_1-score_list_1.min(), '.')

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_1))

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
ax.plot(grad_b_freq_list, score_list_2-score_list_2.min(), '.')

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_2))

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_1))
ax.plot(grad_b_freq_list, fit_func(grad_b_freq_list, *popt_2))